In [3]:
import pandas as pd
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

In [4]:
tweet=pd.read_csv("https://raw.githubusercontent.com/edwardcqian/climate_change_sentiment/master/data/sample_data.csv")

In [5]:
tweet=tweet[['message','sentiment']]

In [6]:
tweet["message_with_stop"] = tweet.message.apply(lambda x: ' '.join(re.sub("(rt )|([0-9]+[0-9]*)|(\#[a-zA-Z]+[a-zA-Z0-9]*)|(@[A-Za-z0-9\_]+)|(:)|(\.)|(,)|(\w+:\/\/\S+)"," ",x.lower()).split()))

In [7]:
tweet_tokenizer = TweetTokenizer()
def remove_stop(tweet):
    tokens = tweet_tokenizer.tokenize(tweet)
    return ' '.join([token for token in tokens if token not in stopwords.words('english')])

In [8]:
tweet["message_without_stop"] = tweet.message_with_stop.apply(remove_stop)

In [9]:
tweet.sample(5)

,message,sentiment,message_with_stop,message_without_stop
30931,"@NBCNews She, a disaster 'expert,' also lied t...",1,she a disaster 'expert ' also lied to congress...,disaster ' expert ' also lied congress said kn...
21852,@MarkHertling @barbarastarrcnn @CNNPolitics Pe...,1,person associates climate change agreement wit...,person associates climate change agreement oba...
39706,RT JaimeJoh: Corporations Will Never Solve Cli...,1,jaimejoh corporations will never solve climate...,jaimejoh corporations never solve climate change
36925,Jeb Bush: To Say Science Is Deciding on Climat...,2,jeb bush to say science is deciding on climate...,jeb bush say science deciding climate change ‘...
7932,You can impact climate change. NZEC is driving...,1,you can impact climate change nzec is driving ...,impact climate change nzec driving zero energy...


In [148]:
tweet["sentiment"].value_counts()

 1    22962
 2     9276
 0     7715
-1     3990
Name: sentiment, dtype: int64

In [149]:
tweet=tweet.loc[tweet['sentiment']!=2]

In [150]:
tweet.sample(5)

,message,sentiment,message_with_stop
567,Exxon Mobil sued over climate change cover-up ...,1,exxon mobil sued over climate change cover-up ...
14771,RT @mitchellvii: If CO2 causes global warming ...,-1,if co causes global warming and the warmer it ...
25254,@FoxNews @KarlRove you are supporting murderer...,1,you are supporting murderers it is murder to f...
32878,We are sleeping through climate change wake-up...,1,we are sleeping through climate change wake-up...
42599,"RT @Josh_Friedman: Also, science has saved my ...",1,also science has saved my life twice go deny c...


In [151]:
tweet["sentiment"].value_counts()

 1    22962
 0     7715
-1     3990
Name: sentiment, dtype: int64

In [152]:
tweet['encoded_cat'] = tweet.sentiment.astype('category').cat.codes

In [153]:
tweet.sample(5)

,message,sentiment,message_with_stop,encoded_cat
14897,RT @JustDaliaAdel: .@EPAScottPruitt doesn't th...,1,doesn't think co drives climate change? let's ...,2
43463,RT @jarrodmckenna: I$q$m waiting for the gover...,0,i$q$m waiting for the government to blame clim...,1
34931,RT @chalu_chokri: Itâ€™s extremely exhausting ...,0,itâ€™s extremely exhausting to tell people abo...,1
1010,RT @NancySinatra: It's time for our leaders to...,1,it's time for our leaders to stop talking abou...,2
32205,RT @nbgmusicyt: fellas we gotta\n\nstop global...,1,fellas we gotta stop global warming,2


In [154]:
tweet.sentiment.value_counts(normalize=True)

 1    0.662359
 0    0.222546
-1    0.115095
Name: sentiment, dtype: float64

In [155]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34667 entries, 0 to 43942
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   message            34667 non-null  object
 1   sentiment          34667 non-null  int64 
 2   message_with_stop  34667 non-null  object
 3   encoded_cat        34667 non-null  int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 1.1+ MB


In [156]:
X=list(tweet['message_without_stop'])

In [157]:
y=list(tweet['encoded_cat'])

In [158]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42,stratify=y)
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size = 0.30, random_state = 142,stratify=y_train)



In [159]:
#!pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [160]:
train_encodings = tokenizer(X_train_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)

In [161]:
import tensorflow as tf

In [20]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))




In [21]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [22]:
from transformers import TFDistilBertForSequenceClassification

#We classify two labels in this example. In case of multiclass 
# classification, adjust num_labels value
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
   4/1214 [..............................] - ETA: 2:42:57 - loss: 1.0975 - accuracy: 0.3711

In [ ]:
model.save_pretrained("/tmp/stance_custom_model")

In [27]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0
